In [1]:
import pandas as pd
import nltk
from nltk.corpus import wordnet
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
import glob
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta
from collections import Counter
stop_words = set(stopwords.words('english')) 
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.stem import PorterStemmer as ps
from nltk.tokenize import word_tokenize
stemmer = ps()
from nltk.stem.cistem import Cistem
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

In [2]:
DATA = '/Users/jane/Desktop/search+youtube_v4/8_41.csv'

df = pd.read_csv(DATA)
data = df[(df['months'] == 3) | (df['months'] == 4) | (df['months'] == 5) & (df['years'] == 2020) ].dropna(subset=['query'])

data['query'].tolist()
    

    
            

['flights from BOS to LIM on Mar 6, 2020 returning Mar 16, 2020',
 'salsa antigua',
 'una noche de copas una noche loca',
 'Maria Conchita Alonso - Noche De Copas - YouTube',
 'Salsa Sensual MIX VOL. 3 [De los 80] (UNA HORA COMPLETA DE ...',
 'greyhound',
 'buffalo',
 'Rochester to Buffalo - 2 ways to travel via bus, and car - Rome2rio',
 'how to go from rochester to buffalo',
 'The Trump Administration Is Denying H-1B Visas at a Dizzying Rate ...',
 'h1b visa extension denial rate',
 'mauricio mesones',
 'Grupo para conocer peruanos en Rochester, expatriados peruanos ...',
 'https://www.yelp.com/search?find_desc=Peruvian+Food&find_loc=Rochester%2C+NY',
 'peruanos en rochester',
 'indignada translate',
 'queso panela',
 'fabian caceres chile',
 'Jetblue Cookies - Belgian Boys',
 'stroopwafel jetblue',
 'dell inspiron headphone jack not working',
 'Headphones jack not working - Dell Community',
 'Solved: Inspiron 15-7???, not detecting headphones - Dell Community',
 'vicks mentholatum',

In [31]:
def tokenize_stem(df):
    searches = df['query'].tolist()
    tokens = []
    punct_list ={'-',':','&','|', 'https', 'http', '...', ',', '?', '+', '(', ')', '*', '$', '@', '%', '^', '~'}
    for q in searches:
        sublist = word_tokenize(q)
        for token in sublist:
            token = token.lower()
            token = stemmer.stem(token)
            if token not in stop_words | punct_list:
                tokens.append(token)
    return tokens

def tokenize_list(searches):
    tokens = []
    punct_list ={'-',':','&','|', 'https', 'http', '...', ',', '?', '+', '(', ')', '*', '$', '@', '%', '^', '~'}
    for q in searches:
        sublist = word_tokenize(q)
        for token in sublist:
            token = token.lower()
            token = stemmer.stem(token)
            if token not in stop_words | punct_list:
                tokens.append(token)
    return tokens

                  
                  
def top_ten(searchlist):
    df = pd.Series(searchlist).value_counts().sort_values(ascending = False).to_frame().reset_index()
    df_new = df.rename(columns={'index':'words'})
    df_new = df_new.head(10)
    return df_new

In [32]:
df_new = top_ten(tokenize_stem(data))

def string_list(df):
    return df_new['words'].tolist()
def make_dict(listofStr):
    return { i : listOfStr[i] for i in range(0, len(listOfStr) ) }
def make_dict2(listofStr):
    dict_1 = make_dict(listofStr)
    return { v : k for k, v in dict_1.items()}
    

['2020', 'coronaviru', 'de', 'map', 'flight', 'return', 'hopkin', 'john', 'lim', 'mba']


In [36]:
data['datetime']= pd.to_datetime(data['datetime'])
dictOfWords = make_dict(listOfStr)
dictOfWords2 = make_dict2(listOfStr)

history = []
weekly_totals = []
labels = [] 
for week, wdf in data.set_index('datetime').groupby(pd.Grouper(freq='W')):
    if len(wdf) >0:
        wdist = [0 for i in range(10)]
        week_label = str(week).split(' ')[0]
        labels.append(week_label)
        weekly_totals.append(len(wdf))
        weekly_queries = [str(q).lower() for q in wdf['query'].values.tolist()]
        words = tokenize_list(weekly_queries)
        for w in words:
            if w in dictOfWords2:
                wdist[dictOfWords2[w]] = wdist[dictOfWords2[w]]+ 1
        history.append(wdist)
print(history)

{0: '2020', 1: 'coronaviru', 2: 'de', 3: 'map', 4: 'flight', 5: 'return', 6: 'hopkin', 7: 'john', 8: 'lim', 9: 'mba'}
[[2, 0, 4, 0, 1, 1, 0, 0, 1, 0], [27, 2, 3, 4, 17, 13, 0, 0, 12, 0], [10, 28, 4, 4, 5, 5, 0, 0, 4, 0], [31, 18, 8, 0, 16, 15, 0, 0, 13, 0], [10, 26, 2, 10, 2, 2, 8, 8, 2, 0], [19, 24, 7, 25, 9, 9, 24, 24, 9, 0], [42, 33, 12, 26, 20, 19, 26, 26, 14, 2], [0, 14, 5, 12, 0, 0, 12, 12, 0, 7], [0, 0, 11, 0, 0, 0, 0, 0, 0, 0], [16, 1, 4, 0, 7, 7, 0, 0, 6, 5], [0, 0, 5, 0, 0, 0, 0, 0, 0, 7], [18, 1, 12, 1, 8, 7, 0, 0, 0, 33], [1, 1, 13, 2, 0, 0, 0, 0, 0, 3], [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]]
